In [2]:
%pylab inline

from __future__ import print_function
from __future__ import division

import sklearn
import numpy
import scipy
import pandas
import matplotlib
import seaborn

import json
import re

from matplotlib import pyplot
from collections import Counter

from sklearn.feature_extraction.text import CountVectorizer

Populating the interactive namespace from numpy and matplotlib


In [3]:
def getlinesep(mail):
    lineseps = ['\r\n', '\n']
    return next(x for x in lineseps if 2 * x in mail)

def parse(mail):
    ls = getlinesep(mail)
    bs = 2 * ls
    
    header, body = mail.split(bs, 1)
    header = header.replace(ls + '\t', ' ').replace(ls + ' ', ' ')
    
    header = [(str(x).replace('_', '--'), y) for x, y in [z.split(':', 1) for z in header.split(ls)]]
    body = ('body', body)
    
    return header + [body] + [('my_linesep', ls)]

def try_parse(mail):
    try:
        return parse(mail)
    except ValueError:
        return {'my_unparseable': True, 'my_reason': 'badheader'}
    except StopIteration:
        return {'my_unparseable': True, 'my_reason': 'nosep'}
    


In [33]:
vectorizer = CountVectorizer(min_df=1)
corpus = [
     'This is the first document.',
     'This is the second second document.',
     'And the third one.',
     'Is this the first document?',
]
X = vectorizer.fit_transform(corpus)
analyze = vectorizer.build_analyzer()
analyze("This is a text document to analyze.") 
last1 = vectorizer.get_feature_names()
last2 = X.toarray()
print(last1)
print(last2)

[u'and', u'document', u'first', u'is', u'one', u'second', u'the', u'third', u'this']
[[0 1 1 1 0 0 1 0 1]
 [0 1 0 1 0 2 1 0 1]
 [1 0 0 0 1 0 1 1 0]
 [0 1 1 1 0 0 1 0 1]]


In [117]:
data = pandas.read_csv('data/train.csv', usecols = ['body', 'spam']).fillna('')

# Count Vectorizer

In [149]:
data.index.size

81000

In [215]:
vec = CountVectorizer(token_pattern = r'[a-z]{4,}', min_df = 800)
counts = vec.fit_transform(data.body)

In [216]:
appareances = counts.minimum(1)

In [217]:
Pw = appareances.mean(axis = 0)

In [218]:
Pws = appareances[numpy.where(data.spam)].mean(axis = 0)

In [219]:
Ps = data.spam.mean()

In [220]:
Psw = Pws * Ps / Pw

In [221]:
counts.shape

(81000, 1649)

In [225]:
numpy.sort(Psw)

matrix([[ 0.,  0.,  0., ...,  1.,  1.,  1.]])

In [228]:
numpy.sort(numpy.asarray(Psw)[0])

array([ 0.,  0.,  0., ...,  1.,  1.,  1.])

In [210]:
numpy.array(vec.get_feature_names())[[numpy.argsort(numpy.asarray(Psw)[0])[::-1][:10]]]

array([u'dcenter', u'corel', u'dleft', u'rfkindysadvnqw', u'nerasdf',
       u'dreamweaver', u'dbaseline', u'cialis', u'dwindows', u'imageready'], 
      dtype='<U15')

In [224]:
vec.get_feature_names()[numpy.argmax(Psw)]

u'cialis'

In [77]:
type(appareances)

scipy.sparse.csr.csr_matrix

In [84]:
data['spam'] = data.index > 50

In [89]:
q = appareances[data.spam]

In [86]:
appareances.shape

(100, 4108)

In [91]:
data.spam.sum()

49

In [93]:
data.spam.shape

(100,)

In [104]:
numpy.where(appareances)

(49, 4108)

In [109]:
numpy.asarray(appareances.todense())[data.spam == True].shape

/usr/local/lib/python2.7/site-packages/ipykernel/__main__.py:1: FutureWarning: in the future, boolean array-likes will be handled as a boolean array index
  if __name__ == '__main__':


(100, 4108)

In [ ]:
Pws = appareances[data.spam].mean(axis = 0)

In [112]:
numpy.where(data.spam)[0]

array([51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67,
       68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84,
       85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99])

In [115]:
appareances[numpy.where(data.spam)]

<49x4108 sparse matrix of type '<type 'numpy.int64'>'
	with 5504 stored elements in Compressed Sparse Row format>

In [66]:
appearences

<10x671 sparse matrix of type '<type 'numpy.int64'>'
	with 853 stored elements in Compressed Sparse Row format>

# 100 palabras más spammeras

In [240]:
data = pandas.read_csv('data/train.csv', usecols = ['body', 'spam']).fillna('')

In [255]:
data.head().body.str.replace(r'<[^>]*>', ' ')

0    chris,  thanks for the invitation. the evening...
1     \t  \t  \t  \t  \t  \t  \t  \t  \t  \t  \t  \...
2    we have determined (vickers and tycholiz and h...
3    yes,  it is a great accomplishment.  please do...
4    fyi  ---------------------- forwarded by richa...
Name: body, dtype: object

In [267]:
vec = CountVectorizer(token_pattern = r'[a-z]{4,}', min_df = 800)

data_norm = data.body.str.replace('<NL>', ' ').replace(r'\\', ' ').str.replace(r'\"', ' ').str.replace(r'"[^"]*"', ' ').str.replace(r'<[^>]*>', ' ')
counts = vec.fit_transform(data_norm)

In [268]:
appareances = counts.minimum(1)
Pw = appareances.mean(axis = 0)
Pws = appareances[numpy.where(data.spam)].mean(axis = 0)
Ps = data.spam.mean()
Psw = Pws * Ps / Pw

In [266]:
data.body[1057]

'<b>power markets 2002: <br>risk and reward</b> (<b>april 17-18, 2002  las vegas, nv</b>)<NL> <NL>sponsored by <b>the power marketing association</b><br>download brochure at: <a href="http://www.pmaconference.com/markets2002.pdf">http://www.pmaconference.com/markets2002.pdf</a> <NL><NL><b>attend the power marketing association\'s seventh <NL>annual spring conference:  power markets 2002: risk <NL>and reward, april 17-18, 2002, at the excalibur hotel in <NL>las vegas, nevada.</b><p><NL><NL>we will see record numbers of new powerplants coming <NL>on line this year. new, much larger rtos promise <NL>dramatic increases in liquidity. the ##### fallout will <NL>continue to force balance sheet improvements-- perhaps <NL>more mergers and acquisitions. price volatility will <NL>continue, and risk management techniques will continue <NL>to be honed. <p><NL><NL>come hear the experts who will help you survive and <NL>thrive in 2002! <NL><NL><NL><NL><NL><NL>topics<NL><NL><b>transmission and rtos:</

In [272]:
map(print, data_norm[data_norm.str.contains(' weight ')].head())

#####'s lay turns down severance pay the wall street journal, 11/14/01 ##### ceo cut stakes in firms where he sits in director's seat the wall street journal, 11/14/01 ##### chief will give up severance the new york times, 11/14/01 companies & finance the americas - regulators face new boundaries in ##### deal. financial times, 11/14/01 front page - companies & markets: ##### chief will turn down dollars 60m takeover windfall  financial times; nov 14, 2001  ##### chief relinquishes severance pay los angeles times, 11/14/01 ##### ceo says no to $60.6 million the washington post, 11/14/01 #####'s lay sells shareholdings in other companies, wsj reports bloomberg, 11/14/01  greenspan takes on energy at rice award presentation houston chronicle, 11/14/01 usa: #####'s lay says no to hefty pay package. reuters english news service, 11/14/01 ##### chairman lay decides to forgo $60.6 million severance payment dow jones business news, 11/14/01 ##### chairman says he won't take dlrs 60 million co

[None, None, None, None, None]

In [269]:
numpy.array(vec.get_feature_names())[[numpy.argsort(numpy.asarray(Psw)[0])[::-1][:100]]]

array([u'dreamweaver', u'cialis', u'nerasdf', u'rfkindysadvnqw', u'corel',
       u'bgcolor', u'illustrator', u'photoshop', u'diso', u'href',
       u'macromedia', u'cellpadding', u'viagra', u'pills', u'serif',
       u'helvetica', u'fireworks', u'verdana', u'arial', u'meds',
       u'prescription', u'discreet', u'ffffff', u'dose', u'pill', u'width',
       u'sans', u'quot', u'studio', u'premiere', u'foresee', u'height',
       u'multi', u'beliefs', u'specials', u'font', u'align', u'tabs',
       u'thousand', u'advises', u'nbsp', u'adobe', u'acrobat', u'drugs',
       u'images', u'anticipates', u'hundred', u'inherent', u'predictions',
       u'mime', u'color', u'mailings', u'weight', u'flash',
       u'uncertainties', u'format', u'differ', u'generic', u'soft',
       u'advertisement', u'materially', u'solicitation', u'websites',
       u'style', u'cheaper', u'deciding', u'presently', u'graphics',
       u'speculative', u'pack', u'pertaining', u'creative', u'advisor',
       u'symbol', 

# Viejo

In [73]:
def NL_to_spaces(s):
    return s.replace("<NL>", " ")

In [238]:
array = numpy.asarray([5,4,3,2,1])
indexes = numpy.argsort(array)
array
indexes
array[indexes]

array([1, 2, 3, 4, 5])